In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import seaborn as sns
import pickle
from tqdm import tqdm
import math
import os
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import pickle

In [2]:
file= 'E:\CIT\MSc_AI\Research project_NIMBUS\Data\working_data\data_2019.csv'
data = pd.read_csv(file,infer_datetime_format=True,index_col=0)
data['applicable_date'] = pd.to_datetime(data['applicable_date'],format='%Y-%m-%d %H:%M:%S')
print(data.shape)
data.head()

(7416, 16)


,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2019-01-01 00:00:00,70.28,0.0,8.8,4.626,250,0.0,8.5,2.056,240,0.0,9.9,2.056,260,59.41,2.62
1,2019-01-01 01:00:00,65.18,0.0,8.3,4.626,250,0.0,8.5,1.542,270,0.0,9.3,2.056,240,59.41,2.62
2,2019-01-01 02:00:00,60.91,0.0,8.4,4.112,240,0.0,8.4,1.542,280,0.0,9.2,2.570,240,59.41,2.62
3,2019-01-01 03:00:00,65.16,0.0,8.6,4.112,250,0.0,8.5,2.570,260,0.0,9.1,2.570,250,59.41,2.62
4,2019-01-01 04:00:00,65.16,0.0,8.4,5.140,250,0.0,8.5,2.570,270,0.0,8.8,2.056,250,59.41,2.62


In [3]:
data['applicable_date'].min(), data['applicable_date'].max(), (data['applicable_date'].max() - data['applicable_date'].min())

(Timestamp('2019-01-01 00:00:00'),
 Timestamp('2019-12-12 22:00:00'),
 Timedelta('345 days 22:00:00'))

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7416 entries, 0 to 7415
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   applicable_date    7416 non-null   datetime64[ns]
 1   elec_prices        7416 non-null   float64       
 2   rain_d             7416 non-null   float64       
 3   temp_d             7416 non-null   float64       
 4   wdsp_d             7416 non-null   float64       
 5   wddir_d            7416 non-null   int64         
 6   rain_c             7416 non-null   float64       
 7   temp_c             7416 non-null   float64       
 8   wdsp_c             7416 non-null   float64       
 9   wddir_c            7416 non-null   int64         
 10  rain_g             7416 non-null   float64       
 11  temp_g             7416 non-null   float64       
 12  wdsp_g             7416 non-null   float64       
 13  wddir_g            7416 non-null   int64         
 14  daily_oi

In [5]:
file= 'E:\CIT\MSc_AI\Research project_NIMBUS\Data\working_data\data_2018_19.csv'
data_1819 = pd.read_csv(file,infer_datetime_format=True,index_col=0)
data_1819['applicable_date'] = pd.to_datetime(data_1819['applicable_date'],format='%Y-%m-%d %H:%M:%S')
print(data_1819.shape)
data_1819.head()

(9686, 16)


,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2018-09-30 23:00:00,71.267,0.0,5.4,5.140,280,0.0,8.1,3.598,320,0.0,3.9,1.542,290,78.89,2.79
1,2018-10-01 00:00:00,67.212,0.0,5.2,5.140,280,0.0,7.7,4.112,320,0.0,6.2,1.542,270,84.94,2.80
2,2018-10-01 01:00:00,60.500,0.0,5.2,5.654,280,0.0,7.4,3.084,320,0.0,7.2,2.056,270,84.94,2.80
3,2018-10-01 02:00:00,63.682,0.0,4.8,4.626,270,0.0,5.3,1.542,200,0.0,6.9,2.056,280,84.94,2.80
4,2018-10-01 03:00:00,71.617,0.0,4.6,3.084,260,0.0,6.5,1.542,290,0.0,7.2,2.056,280,84.94,2.80


In [6]:
data_1819['applicable_date'].min(), data_1819['applicable_date'].max(), (data_1819['applicable_date'].max() - data_1819['applicable_date'].min())

(Timestamp('2018-09-30 23:00:00'),
 Timestamp('2019-12-12 22:00:00'),
 Timedelta('437 days 23:00:00'))

In [7]:
def normalize(x_train,x_test):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler().fit(x_train)
    x_train_norm = scaler.transform(x_train)
    x_test_norm = scaler.transform(x_test)
    return x_train_norm,x_test_norm

In [8]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.mean(y_true))) * 100
def eval_metrics(y_true,y_pred):
    MAE = mean_absolute_error(y_true,y_pred)
    MSE = mean_squared_error(y_true,y_pred)
    RMSE = mean_squared_error(y_true,y_pred,squared=False)
    MAPE = mean_absolute_percentage_error(y_true,y_pred)
    R2 = r2_score(y_true, y_pred)
    print("MAE: ",round((MAE),4))
    print("MSE: ",round((MSE),4))
    print("RMSE: ",round((RMSE),4))
    print("MAPE: ",round((MAPE),4))
    print("Coefficient of determination (R-squared): ",round((R2),4))
    print('\n')
    return MAE,MSE,RMSE,MAPE,R2

In [9]:
def multi_step(data,target):

    for time_step in range(1,24):
        name = 't'+'+'+str(time_step)
        data[name] = data[target].shift(-time_step)

    return data

In [10]:
def calc_24lags(data,feature):
    if feature == 'elec_prices':
        for time_step in range(1,25):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
    else:
        for time_step in range(1,24):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
            
    return data

In [11]:
def remove_23rows_hour_col(data):
    data = data[data['Hour']==23]
    data= data.drop(['Hour'], axis=1)
    data = data.reset_index(drop=True)
    print(data.shape)
    return data

In [12]:
#80-20split
def train_test_split_tbs(final_data):
    train,test = final_data.iloc[:int(0.8*final_data.shape[0]), :] , final_data.iloc[int(0.8*final_data.shape[0]):, :]
    print(train.shape)
    print(test.shape)
    
    return train,test

In [13]:
def GBM_training(X,y):

    start_time = datetime.now()

    wrapper1 = MultiOutputRegressor(GradientBoostingRegressor(max_features='sqrt', random_state=100))

    hyperparameters = {'estimator__learning_rate':[0.1,0.05,0.025,0.0125],'estimator__n_estimators':range(20,101,10),'estimator__max_depth':range(5,14,2),
                       'estimator__min_samples_split':range(100,501,100), 'estimator__max_features':range(1,7,1), 'estimator__min_samples_leaf':range(10,61,10),
                       'estimator__subsample': [0.5,0.6,0.7,0.75,0.8,0.85,0.9]}

    rsearch = RandomizedSearchCV(estimator = wrapper1, param_distributions = hyperparameters, scoring='neg_mean_absolute_error',
                                  n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("\n\n")
    print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)

In [14]:
def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
    
    learning_rate=learning_rate
    n_estimators=n_estimators
    max_depth= max_depth
    min_samples_split= min_samples_split
    max_features= max_features
    min_samples_leaf= min_samples_leaf
    subsample= subsample

    start_time = datetime.now()

    GBM = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=learning_rate, n_estimators=n_estimators, 
                                                         max_depth= max_depth,min_samples_split= min_samples_split,
                                                         max_features= max_features, min_samples_leaf= min_samples_leaf,
                                                         subsample= subsample, random_state=100))

    GBM.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

    return GBM

In [15]:
def XGBM_training(X,y):

    start_time = datetime.now()

    wrapper = MultiOutputRegressor(XGBRegressor(n_jobs = -1))

    hyperparameters = {'estimator__learning_rate':[0.2,0.1,0.05,0.025,0.0125],'estimator__n_estimators':range(50, 500, 50),
                       'estimator__max_depth':range(3,14,2), 'estimator__subsample':np.arange(0.1, 1.1, 0.1),
                       'estimator__colsample_bytree':np.arange(0.1, 1.1, 0.1)}

    rsearch = RandomizedSearchCV(estimator = wrapper, param_distributions = hyperparameters, scoring='neg_mean_absolute_error',
                                  n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("\n\n")
    print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)

In [16]:
def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

    learning_rate = learning_rate
    n_estimators = n_estimators
    max_depth = max_depth
    subsample = subsample
    colsample_bytree = colsample_bytree

    start_time = datetime.now()

    XGBM = MultiOutputRegressor(XGBRegressor(learning_rate=learning_rate, n_estimators=n_estimators, max_depth= max_depth,
                                            subsample= subsample, colsample_bytree = colsample_bytree, random_state=100))

    XGBM.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

    return XGBM

In [17]:
def LGBM_training(X,y):

    start_time = datetime.now()

    wrapper = MultiOutputRegressor(LGBMRegressor(random_state=100, n_jobs = -1))

    hyperparameters = {'estimator__learning_rate':[0.2,0.1,0.05,0.025,0.0125],'estimator__n_estimators':range(50, 501, 50),
                       'estimator__max_depth':range(3,14,2), 'estimator__subsample':np.arange(0.1, 1.1, 0.1),
                       'estimator__colsample_bytree':np.arange(0.1, 1.1, 0.1), 'estimator__num_leaves':[round(0.6*2**x) for x in range(3,14,2)],
                      'estimator__min_child_samples ':range(10,71,10)}

    rsearch = RandomizedSearchCV(estimator = wrapper, param_distributions = hyperparameters, scoring='neg_mean_absolute_error',
                                  n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("\n\n")
    print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)

In [18]:
def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

    num_leaves = num_leaves
    min_child_samples = min_child_samples
    learning_rate = learning_rate
    n_estimators = n_estimators
    max_depth= max_depth
    subsample= subsample
    colsample_bytree = colsample_bytree

    start_time = datetime.now()

    LGB = MultiOutputRegressor(LGBMRegressor(num_leaves=num_leaves,min_child_samples = min_child_samples,learning_rate=learning_rate,n_estimators=n_estimators,
                                               max_depth= max_depth,subsample= subsample,colsample_bytree = colsample_bytree,random_state=100,
                                               n_jobs = -1))

    LGB.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return LGB

In [19]:
def random_splitting(data):
    from sklearn.model_selection import train_test_split
    X= data[x_names]; y= data[y_names]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)
    print(X_train.shape,X_val.shape,X_test.shape)
    print(y_train.shape,y_val.shape,y_test.shape)
    
    return X_train,X_val,X_test,y_train,y_val, y_test

In [20]:
def testing_runs(n,data,model):
    mae = []; mse = []; rmse = []; mape = []; r2 = []
    for i in range(n):
        print("Round {}: ".format(i+1))
        X_train,X_val,X_test,y_train,y_val, y_test = random_splitting(data)
        x_train_norm,x_test_norm = normalize(X_train,X_test)
        y_pred= model.predict(x_test_norm)
        MAE,MSE,RMSE,MAPE,R2 = eval_metrics(y_test,y_pred)
        mae.append(round((MAE),4))
        mse.append(round(MSE,4))
        rmse.append(round(RMSE,4))
        mape.append(round(MAPE,4))
        r2.append(round(R2,4))
    print("Average MAE: ",round(np.mean(mae),4))
    print("Average MSE: ",round(np.mean(mse),4))
    print("Average RMSE: ", round(np.mean(rmse),4))
    print("Average MAPE: ", round(np.mean(mape),4))
    print("Average R2: ", round(np.mean(r2),4))
    
    return mae,mse,rmse,mape,r2

In [25]:
def feature_pipeline(data,option):
    
    if option==1:

        df= data[['applicable_date','elec_prices']]
        df['expanding_window_price'] = df['elec_prices'].expanding().mean()
        wdsp_sum= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g']).values
        wdsp_sum_sqrt= np.sqrt(wdsp_sum)
        df['wdsp_sum_sqrt'] = wdsp_sum_sqrt
        wdsp_sqr= (data['wdsp_d']*data['wdsp_d']+data['wdsp_c']*data['wdsp_c']+data['wdsp_g']*data['wdsp_g']).values
        df['wdsp_sqr'] = wdsp_sqr
        df['wdsp_g'] = data['wdsp_g']
        df['wdsp_c'] = data['wdsp_c']
        df['Hour'] = df['applicable_date'].dt.hour
    
    elif option==2:

        df= data[['applicable_date','elec_prices']]
        df['expanding_window_price'] = df['elec_prices'].expanding().mean()
        wdsp_sum= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g']).values
        wdsp_sum_sqrt= np.sqrt(wdsp_sum)
        df['wdsp_sum_sqrt'] = wdsp_sum_sqrt
        wdsp_sqr= (data['wdsp_d']*data['wdsp_d']+data['wdsp_c']*data['wdsp_c']+data['wdsp_g']*data['wdsp_g']).values
        df['wdsp_sqr'] = wdsp_sqr
        df['wdsp_g'] = data['wdsp_g']
        df['wdsp_c'] = data['wdsp_c']
        df['wdsp_sum'] = wdsp_sum
        ng = data.daily_natural_gas.values
        ng_log = np.log(ng)
        df['ng_log'] = ng_log
        df['daily_natural_gas'] = ng
        df['ng_sqr'] = ng*ng
        df['wdsp_sum_log'] = np.log(wdsp_sum)
        df['ng_sqrt'] = np.sqrt(ng)
        df['wdsp_d'] = data['wdsp_d']
        df['Hour'] = df['applicable_date'].dt.hour
    
    return df

In [24]:
data_1= data.copy()
data_1 = feature_pipeline(data_1,1)
print(data_1.shape)
print(data_1.columns)
data_1.head()

(7416, 8)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'Hour'],
      dtype='object')


,applicable_date,elec_prices,expanding_window_price,wdsp_sum_sqrt,wdsp_sqr,wdsp_g,wdsp_c,Hour
0,2019-01-01 00:00:00,70.28,70.280000,2.956011,29.854148,2.056,2.056,0
1,2019-01-01 01:00:00,65.18,67.730000,2.867752,28.004776,2.056,1.542,1
2,2019-01-01 02:00:00,60.91,65.456667,2.867752,25.891208,2.570,1.542,2
3,2019-01-01 03:00:00,65.16,65.382500,3.041710,30.118344,2.570,2.570,3
4,2019-01-01 04:00:00,65.16,65.338000,3.125060,37.251636,2.056,2.570,4


In [25]:
data_1.isnull().values.any()

False

In [26]:
data_1 = multi_step(data_1,'elec_prices')

data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_sum_sqrt')
data_1 = calc_24lags(data_1,'wdsp_sqr')
data_1 = calc_24lags(data_1,'wdsp_g')
data_1 = calc_24lags(data_1,'wdsp_c')

In [27]:
print(data_1.shape)

(7416, 147)


In [28]:
print(data_1.columns)

Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'Hour', 't+1', 't+2',
       ...
       'lag_14_wdsp_c', 'lag_15_wdsp_c', 'lag_16_wdsp_c', 'lag_17_wdsp_c',
       'lag_18_wdsp_c', 'lag_19_wdsp_c', 'lag_20_wdsp_c', 'lag_21_wdsp_c',
       'lag_22_wdsp_c', 'lag_23_wdsp_c'],
      dtype='object', length=147)


In [29]:
data_1.isnull().values.any()

True

In [30]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)

(7393, 147)


In [31]:
data_1.isnull().values.any()

False

In [32]:
data_1 = remove_23rows_hour_col(data_1)
print(data_1.shape)
train,test = train_test_split_tbs(data_1)
train.columns

(310, 146)
(310, 146)
(248, 146)
(62, 146)


Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 't+1', 't+2', 't+3',
       ...
       'lag_14_wdsp_c', 'lag_15_wdsp_c', 'lag_16_wdsp_c', 'lag_17_wdsp_c',
       'lag_18_wdsp_c', 'lag_19_wdsp_c', 'lag_20_wdsp_c', 'lag_21_wdsp_c',
       'lag_22_wdsp_c', 'lag_23_wdsp_c'],
      dtype='object', length=146)

In [33]:
cols = train.columns.to_list()
cols

['applicable_date',
 'elec_prices',
 'expanding_window_price',
 'wdsp_sum_sqrt',
 'wdsp_sqr',
 'wdsp_g',
 'wdsp_c',
 't+1',
 't+2',
 't+3',
 't+4',
 't+5',
 't+6',
 't+7',
 't+8',
 't+9',
 't+10',
 't+11',
 't+12',
 't+13',
 't+14',
 't+15',
 't+16',
 't+17',
 't+18',
 't+19',
 't+20',
 't+21',
 't+22',
 't+23',
 'lag_1_elec_prices',
 'lag_2_elec_prices',
 'lag_3_elec_prices',
 'lag_4_elec_prices',
 'lag_5_elec_prices',
 'lag_6_elec_prices',
 'lag_7_elec_prices',
 'lag_8_elec_prices',
 'lag_9_elec_prices',
 'lag_10_elec_prices',
 'lag_11_elec_prices',
 'lag_12_elec_prices',
 'lag_13_elec_prices',
 'lag_14_elec_prices',
 'lag_15_elec_prices',
 'lag_16_elec_prices',
 'lag_17_elec_prices',
 'lag_18_elec_prices',
 'lag_19_elec_prices',
 'lag_20_elec_prices',
 'lag_21_elec_prices',
 'lag_22_elec_prices',
 'lag_23_elec_prices',
 'lag_24_elec_prices',
 'lag_1_wdsp_sum_sqrt',
 'lag_2_wdsp_sum_sqrt',
 'lag_3_wdsp_sum_sqrt',
 'lag_4_wdsp_sum_sqrt',
 'lag_5_wdsp_sum_sqrt',
 'lag_6_wdsp_sum_sqrt',

In [34]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:7]
lags = cols[30:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[7:30]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['expanding_window_price', 'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_sum_sqrt', 'lag_2_wdsp_sum_sqrt', 'lag_3_wdsp_sum_sqrt', 'lag_4_wdsp_sum_sqrt', 'lag_5_wdsp_sum_sqrt', 'lag_6_wdsp_sum_sqrt', 'lag_7_wdsp_sum_sqrt', 'lag_8_wdsp_sum_sqrt', 'lag_9_wdsp_sum_sqrt', 'lag_10_wdsp_sum_sqrt', 'lag_11_wdsp_sum_sqrt', 'lag_12_wdsp_sum_sqrt', 'lag_13_wdsp_sum_sqrt', 'lag_14_wdsp_sum_sqrt', 'lag_15_wdsp_sum_sqrt', 'lag_16_wdsp_sum_sqrt', 'lag_17_wdsp_sum_sqrt', 'lag_1

In [35]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(248, 121) (248, 24)
(62, 121) (62, 24)


In [36]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [37]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:15.210951



{'mean_fit_time': array([0.6694916 , 0.5858588 , 1.43945463, 0.55820305, 0.96838109,
       0.66356058, 1.41973281, 1.15977297, 0.87923071, 1.23277857]), 'std_fit_time': array([0.0118672 , 0.01196895, 0.02841045, 0.02675397, 0.01372353,
       0.0204413 , 0.01404482, 0.06008278, 0.05423615, 0.09253066]), 'mean_score_time': array([0.01600211, 0.01760213, 0.0176019 , 0.01680224, 0.01920185,
       0.01760237, 0.01600151, 0.02000277, 0.01634426, 0.00960131]), 'std_score_time': array([8.39896915e-07, 3.20044760e-03, 3.20080556e-03, 2.39946860e-03,
       3.92004831e-03, 3.20020942e-03, 1.43764624e-06, 5.36821819e-03,
       1.02537592e-03, 3.20180773e-03]), 'param_estimator__subsample': masked_array(data=[0.5, 0.8, 0.75, 0.75, 0.75, 0.8, 0.85, 0.6, 0.75, 0.7],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(

In [39]:
gbm_top5 = GBM_model(x_train_norm,y_train,0.1,30,11,100,4,40,0.8)

Duration: 0:00:00.403701


In [40]:
y_pred= gbm_top5.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.3969
MSE:  213.4318
RMSE:  14.6093
MAPE:  21.6426
Coefficient of determination (R-squared):  0.0763




(10.396890142463684,
 213.43176601561075,
 14.609304090736517,
 21.64258350169268,
 0.0763005645637784)

In [41]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 10
Duration: 0:00:56.795928



{'mean_fit_time': array([ 3.52043417,  3.90332148,  3.17410429,  3.34897418,  3.82094543,
       13.30342908,  2.36625087,  6.25314112,  3.26203978, 12.1609185 ]), 'std_fit_time': array([0.08438113, 0.12588786, 0.12067101, 0.22356601, 0.10441027,
       0.44227492, 0.1309742 , 0.15876194, 0.12438479, 0.41007299]), 'mean_score_time': array([0.0336019 , 0.05356622, 0.03840516, 0.02303448, 0.04320641,
       0.05120604, 0.01953776, 0.04886422, 0.04185965, 0.03875437]), 'std_score_time': array([0.00480059, 0.00380827, 0.00861716, 0.00252347, 0.00391866,
       0.00733168, 0.00377261, 0.00872203, 0.00798997, 0.00902408]), 'param_estimator__subsample': masked_array(data=[0.4, 0.30000000000000004, 0.5, 0.5, 0.6,
                   0.30000000000000004, 0.5, 0.4, 0.4, 0.8],
             mask=[False, False

In [42]:
lgbm_top5 = LGBM_model(x_train_norm,y_train,77,10,0.0125,450,13,0.8,0.7)

Duration: 0:00:07.337317


In [43]:
y_pred= lgbm_top5.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.2784
MSE:  243.3573
RMSE:  15.5999
MAPE:  23.4775
Coefficient of determination (R-squared):  -0.0653




(11.278357225225259,
 243.3572910981269,
 15.5999131759804,
 23.4774807335817,
 -0.06528035254547165)

In [44]:
XGBM_training(x_train_norm,y_train)

Duration: 0:02:18.281596



{'mean_fit_time': array([ 6.61567538, 11.75961037, 27.5886795 , 21.41200995,  7.58293605,
        7.33008196, 13.64985456, 23.59260798, 24.53235352,  8.15477617]), 'std_fit_time': array([0.04714212, 1.86585422, 0.91629659, 1.1393975 , 0.6962971 ,
       0.1992324 , 0.18335201, 0.28183055, 0.96838214, 0.50978076]), 'mean_score_time': array([0.0431253 , 0.06400926, 0.12323112, 0.06376877, 0.04285216,
       0.05080237, 0.03827832, 0.05004981, 0.04307327, 0.0378967 ]), 'std_score_time': array([0.00252592, 0.00876663, 0.02090655, 0.00900559, 0.00684522,
       0.00305668, 0.00608299, 0.00763547, 0.01356349, 0.00728736]), 'param_estimator__subsample': masked_array(data=[0.1, 0.30000000000000004, 0.6, 0.5, 0.1, 0.5, 0.8, 0.5,
                   0.30000000000000004, 0.4],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': mask

In [45]:
xgb_top5 = XGBM_model(x_train_norm,y_train,0.025,250,3,0.4,0.4)

Duration: 0:00:02.705201


In [46]:
y_pred= xgb_top5.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.4861
MSE:  216.3746
RMSE:  14.7097
MAPE:  21.8282
Coefficient of determination (R-squared):  0.0692




(10.486054378287125,
 216.37460239757786,
 14.709677168367016,
 21.828191351033308,
 0.06919317920284224)

### Testing on 2018-2019 data

In [47]:
def feature_pipeline_2(df,data_1819_1819):
    

    df= data_1819[['applicable_date','elec_prices']]
    df['expanding_window_price'] = df['elec_prices'].expanding().mean()
    wdsp_sum= (data_1819['wdsp_d']+data_1819['wdsp_c']+data_1819['wdsp_g']).values
    wdsp_sum_sqrt= np.sqrt(wdsp_sum)
    df['wdsp_sum_sqrt'] = wdsp_sum_sqrt
    wdsp_sqr= (data_1819['wdsp_d']*data_1819['wdsp_d']+data_1819['wdsp_c']*data_1819['wdsp_c']+data_1819['wdsp_g']*data_1819['wdsp_g']).values
    df['wdsp_sqr'] = wdsp_sqr
    df['wdsp_g'] = data_1819['wdsp_g']
    df['wdsp_c'] = data_1819['wdsp_c']
    df['Hour'] = df['applicable_date'].dt.hour
    
    return df

In [48]:
data_test = data_1819.copy()
data_test = feature_pipeline_2(data_test,data_1819)
print(data_test.shape)
print(data_test.columns)
data_test.head()

(9686, 8)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'Hour'],
      dtype='object')


,applicable_date,elec_prices,expanding_window_price,wdsp_sum_sqrt,wdsp_sqr,wdsp_g,wdsp_c,Hour
0,2018-09-30 23:00:00,71.267,71.267000,3.206244,41.742968,1.542,3.598,23
1,2018-10-01 00:00:00,67.212,69.239500,3.285422,45.705908,1.542,4.112,0
2,2018-10-01 01:00:00,60.500,66.326333,3.285422,45.705908,2.056,3.084,1
3,2018-10-01 02:00:00,63.682,65.665250,2.867752,28.004776,2.056,1.542,2
4,2018-10-01 03:00:00,71.617,66.855600,2.584956,16.115956,2.056,1.542,3


In [49]:
data_test = multi_step(data_test,'elec_prices')

data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_sum_sqrt')
data_test = calc_24lags(data_test,'wdsp_sqr')
data_test = calc_24lags(data_test,'wdsp_g')
data_test = calc_24lags(data_test,'wdsp_c')

In [50]:
print(data_test.shape)
print(data_test.columns)
data_test.head()

(9686, 147)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'Hour', 't+1', 't+2',
       ...
       'lag_14_wdsp_c', 'lag_15_wdsp_c', 'lag_16_wdsp_c', 'lag_17_wdsp_c',
       'lag_18_wdsp_c', 'lag_19_wdsp_c', 'lag_20_wdsp_c', 'lag_21_wdsp_c',
       'lag_22_wdsp_c', 'lag_23_wdsp_c'],
      dtype='object', length=147)


,applicable_date,elec_prices,expanding_window_price,wdsp_sum_sqrt,wdsp_sqr,wdsp_g,wdsp_c,Hour,t+1,t+2,...,lag_14_wdsp_c,lag_15_wdsp_c,lag_16_wdsp_c,lag_17_wdsp_c,lag_18_wdsp_c,lag_19_wdsp_c,lag_20_wdsp_c,lag_21_wdsp_c,lag_22_wdsp_c,lag_23_wdsp_c
0,2018-09-30 23:00:00,71.267,71.267000,3.206244,41.742968,1.542,3.598,23,67.212,60.500,...,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598
1,2018-10-01 00:00:00,67.212,69.239500,3.285422,45.705908,1.542,4.112,0,60.500,63.682,...,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598
2,2018-10-01 01:00:00,60.500,66.326333,3.285422,45.705908,2.056,3.084,1,63.682,71.617,...,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598
3,2018-10-01 02:00:00,63.682,65.665250,2.867752,28.004776,2.056,1.542,2,71.617,72.855,...,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598
4,2018-10-01 03:00:00,71.617,66.855600,2.584956,16.115956,2.056,1.542,3,72.855,76.937,...,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598,3.598


In [51]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(9663, 147)


False

In [52]:
data_test = remove_23rows_hour_col(data_test)

(403, 146)


In [53]:
print(x_names)
print(y_names)

['expanding_window_price', 'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_sum_sqrt', 'lag_2_wdsp_sum_sqrt', 'lag_3_wdsp_sum_sqrt', 'lag_4_wdsp_sum_sqrt', 'lag_5_wdsp_sum_sqrt', 'lag_6_wdsp_sum_sqrt', 'lag_7_wdsp_sum_sqrt', 'lag_8_wdsp_sum_sqrt', 'lag_9_wdsp_sum_sqrt', 'lag_10_wdsp_sum_sqrt', 'lag_11_wdsp_sum_sqrt', 'lag_12_wdsp_sum_sqrt', 'lag_13_wdsp_sum_sqrt', 'lag_14_wdsp_sum_sqrt', 'lag_15_wdsp_sum_sqrt', 'lag_16_wdsp_sum_sqrt', 'lag_17_wdsp_sum_sqrt', 'lag_1

In [62]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.6792
MSE:  425.9763
RMSE:  20.6392
MAPE:  20.348
Coefficient of determination (R-squared):  0.3291


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  13.109
MSE:  669.5116
RMSE:  25.8749
MAPE:  22.6211
Coefficient of determination (R-squared):  0.2818


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.7025
MSE:  325.6037
RMSE:  18.0445
MAPE:  20.4474
Coefficient of determination (R-squared):  0.2612


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  13.7971
MSE:  506.3839
RMSE:  22.503
MAPE:  23.8839
Coefficient of determination (R-squared):  0.2899


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  12.2745
MSE:  438.4773
RMSE:  20.9399
MAPE:  21.927
Coefficient of determination (R-squared):  0.2377


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  13.773
MSE:  550.7495
RMSE:  

In [60]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.8657
MSE:  369.5427
RMSE:  19.2235
MAPE:  18.2254
Coefficient of determination (R-squared):  0.4341


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.2746
MSE:  460.0616
RMSE:  21.449
MAPE:  18.7201
Coefficient of determination (R-squared):  0.5101


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.8785
MSE:  344.0667
RMSE:  18.549
MAPE:  19.3629
Coefficient of determination (R-squared):  0.4566


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.3652
MSE:  363.6978
RMSE:  19.0709
MAPE:  19.1402
Coefficient of determination (R-squared):  0.54


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.4208
MSE:  386.8099
RMSE:  19.6675
MAPE:  19.0354
Coefficient of determination (R-squared):  0.5438


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.1437
MSE:  358.6898
RMSE: 

In [56]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.4908
MSE:  588.6054
RMSE:  24.2612
MAPE:  19.1733
Coefficient of determination (R-squared):  0.5186


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.1255
MSE:  359.2668
RMSE:  18.9543
MAPE:  17.5353
Coefficient of determination (R-squared):  0.4618


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.2381
MSE:  206.953
RMSE:  14.3859
MAPE:  17.3922
Coefficient of determination (R-squared):  0.4882


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.129
MSE:  252.76
RMSE:  15.8984
MAPE:  18.6416
Coefficient of determination (R-squared):  0.4338


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  8.6274
MSE:  160.4953
RMSE:  12.6687
MAPE:  15.6851
Coefficient of determination (R-squared):  0.5917


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.8131
MSE:  336.3875
RMSE:  18

In [66]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.272
MSE:  251.3978
RMSE:  15.8555
MAPE:  18.7435
Coefficient of determination (R-squared):  0.3586


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.8284
MSE:  361.4371
RMSE:  19.0115
MAPE:  17.4437
Coefficient of determination (R-squared):  0.395


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.8778
MSE:  399.0514
RMSE:  19.9763
MAPE:  17.0428
Coefficient of determination (R-squared):  0.5032


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.4601
MSE:  344.3114
RMSE:  18.5556
MAPE:  16.871
Coefficient of determination (R-squared):  0.5789


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.3642
MSE:  229.3106
RMSE:  15.143
MAPE:  17.1001
Coefficient of determination (R-squared):  0.5104


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.7968
MSE:  219.6954
RMSE:  14.8

In [57]:
import pickle
# save the model to disk
pickle.dump(gbm_top5, open('GBM_top5_model.sav', 'wb'))
pickle.dump(xgb_top5, open('XGBM_top5_model.sav', 'wb'))
pickle.dump(lgbm_top5, open('LightGBM_top5_model.sav', 'wb'))

In [71]:
mae

[10.272,
 9.8284,
 9.8778,
 9.4601,
 9.3642,
 9.7968,
 10.4764,
 10.6599,
 8.8176,
 8.774]

In [72]:
result_items = pd.DataFrame(columns=["mae", "mse", "rmse", "mape", "r2"])
result_items['mae'] = mae
result_items['mse'] = mse
result_items['rmse'] = rmse
result_items['mape'] = mape
result_items['r2'] = r2
writer = pd.ExcelWriter("E:/CIT/MSc_AI/Research project_NIMBUS/Data/output.xlsx")
result_items.to_excel(writer, index=False)
writer.close()

### Testing for 30 rounds

In [73]:
#GBM
mae1,mse1,rmse1,mape1,r21 = testing_runs(30,data_test,gbm_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.9984
MSE:  453.3516
RMSE:  21.2921
MAPE:  20.9167
Coefficient of determination (R-squared):  0.3563


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  12.0675
MSE:  369.9222
RMSE:  19.2334
MAPE:  21.9468
Coefficient of determination (R-squared):  0.3659


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  13.0493
MSE:  342.6958
RMSE:  18.512
MAPE:  23.5003
Coefficient of determination (R-squared):  0.333


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  12.5737
MSE:  501.3926
RMSE:  22.3918
MAPE:  22.2512
Coefficient of determination (R-squared):  0.3514


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  13.7154
MSE:  474.6256
RMSE:  21.7859
MAPE:  23.2965
Coefficient of determination (R-squared):  0.2235


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.5235
MSE:  330.1812
RMSE

In [74]:
#XGBM
mae2,mse2,rmse2,mape2,r22 = testing_runs(30,data_test,xgb_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.6174
MSE:  298.6319
RMSE:  17.281
MAPE:  20.6114
Coefficient of determination (R-squared):  0.512


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.8375
MSE:  339.4126
RMSE:  18.4232
MAPE:  16.9548
Coefficient of determination (R-squared):  0.4587


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.2649
MSE:  237.8003
RMSE:  15.4208
MAPE:  18.4335
Coefficient of determination (R-squared):  0.3748


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  11.0018
MSE:  309.4429
RMSE:  17.591
MAPE:  19.0203
Coefficient of determination (R-squared):  0.4863


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  12.1146
MSE:  435.0177
RMSE:  20.8571
MAPE:  21.3538
Coefficient of determination (R-squared):  0.4244


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.6709
MSE:  196.1352
RMSE:  

In [78]:
#light GBM
mae3,mse3,rmse3,mape3,r23 = testing_runs(30,data_test,lgbm_top5)

Round 1: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.8194
MSE:  359.0604
RMSE:  18.9489
MAPE:  18.0147
Coefficient of determination (R-squared):  0.4816


Round 2: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  10.3215
MSE:  318.6833
RMSE:  17.8517
MAPE:  17.3483
Coefficient of determination (R-squared):  0.5087


Round 3: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.6805
MSE:  271.6758
RMSE:  16.4826
MAPE:  17.2579
Coefficient of determination (R-squared):  0.4972


Round 4: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.3702
MSE:  208.1429
RMSE:  14.4272
MAPE:  17.2605
Coefficient of determination (R-squared):  0.5054


Round 5: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  9.5136
MSE:  214.5731
RMSE:  14.6483
MAPE:  16.9195
Coefficient of determination (R-squared):  0.497


Round 6: 
(322, 121) (40, 121) (41, 121)
(322, 24) (40, 24) (41, 24)
MAE:  8.5157
MSE:  155.8809
RMSE:  1

In [88]:
result_items = pd.DataFrame(columns=["mae", "mse", "rmse", "mape", "r2"])
result_items['mae'] = mae3
result_items['mse'] = mse3
result_items['rmse'] = rmse3
result_items['mape'] = mape3
result_items['r2'] = r23
writer = pd.ExcelWriter("E:/CIT/MSc_AI/Research project_NIMBUS/Data/output.xlsx")
result_items.to_excel(writer, index=False)
writer.close()

<h1 style="text-align:center;font-size:40px;" > All 12 imp features</h1>

In [28]:
data_1 = feature_pipeline(data,2)
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(7416, 62)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_sum', 'ng_log',
       'daily_natural_gas', 'ng_sqr', 'wdsp_sum_log', 'ng_sqrt', 'wdsp_d',
       'Hour', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8', 't+9',
       't+10', 't+11', 't+12', 't+13', 't+14', 't+15', 't+16', 't+17', 't+18',
       't+19', 't+20', 't+21', 't+22', 't+23', 'lag_1_elec_prices',
       'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices',
       'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices',
       'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices',
       'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices',
       'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices',
       'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices',
       'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices',
       'lag_23_elec_prices', 'lag_24_elec

,applicable_date,elec_prices,expanding_window_price,wdsp_sum_sqrt,wdsp_sqr,wdsp_g,wdsp_c,wdsp_sum,ng_log,daily_natural_gas,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2019-01-01 00:00:00,70.28,70.280000,2.956011,29.854148,2.056,2.056,8.738,0.963174,2.62,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
1,2019-01-01 01:00:00,65.18,67.730000,2.867752,28.004776,2.056,1.542,8.224,0.963174,2.62,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
2,2019-01-01 02:00:00,60.91,65.456667,2.867752,25.891208,2.570,1.542,8.224,0.963174,2.62,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
3,2019-01-01 03:00:00,65.16,65.382500,3.041710,30.118344,2.570,2.570,9.252,0.963174,2.62,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
4,2019-01-01 04:00:00,65.16,65.338000,3.125060,37.251636,2.056,2.570,9.766,0.963174,2.62,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28


In [29]:
data_1.isnull().values.any()

True

In [31]:
data_1 = calc_24lags(data_1,'wdsp_sum_sqrt')
data_1 = calc_24lags(data_1,'wdsp_sqr')
data_1 = calc_24lags(data_1,'wdsp_g')
data_1 = calc_24lags(data_1,'wdsp_c')
data_1 = calc_24lags(data_1,'wdsp_sum')
data_1 = calc_24lags(data_1,'ng_log')
data_1 = calc_24lags(data_1,'daily_natural_gas')
data_1 = calc_24lags(data_1,'ng_sqr')
data_1 = calc_24lags(data_1,'wdsp_sum_log')
data_1 = calc_24lags(data_1,'ng_sqrt')
data_1 = calc_24lags(data_1,'wdsp_d')

In [32]:
print(data_1.shape)
print(data_1.columns)
data_1.head()

(7416, 315)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_sum', 'ng_log',
       'daily_natural_gas',
       ...
       'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d',
       'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d',
       'lag_22_wdsp_d', 'lag_23_wdsp_d'],
      dtype='object', length=315)


,applicable_date,elec_prices,expanding_window_price,wdsp_sum_sqrt,wdsp_sqr,wdsp_g,wdsp_c,wdsp_sum,ng_log,daily_natural_gas,...,lag_14_wdsp_d,lag_15_wdsp_d,lag_16_wdsp_d,lag_17_wdsp_d,lag_18_wdsp_d,lag_19_wdsp_d,lag_20_wdsp_d,lag_21_wdsp_d,lag_22_wdsp_d,lag_23_wdsp_d
0,2019-01-01 00:00:00,70.28,70.280000,2.956011,29.854148,2.056,2.056,8.738,0.963174,2.62,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
1,2019-01-01 01:00:00,65.18,67.730000,2.867752,28.004776,2.056,1.542,8.224,0.963174,2.62,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
2,2019-01-01 02:00:00,60.91,65.456667,2.867752,25.891208,2.570,1.542,8.224,0.963174,2.62,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
3,2019-01-01 03:00:00,65.16,65.382500,3.041710,30.118344,2.570,2.570,9.252,0.963174,2.62,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
4,2019-01-01 04:00:00,65.16,65.338000,3.125060,37.251636,2.056,2.570,9.766,0.963174,2.62,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626


In [33]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(7393, 315)


False

In [34]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(310, 314)
(248, 314)
(62, 314)


Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_sum', 'ng_log',
       'daily_natural_gas',
       ...
       'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d',
       'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d',
       'lag_22_wdsp_d', 'lag_23_wdsp_d'],
      dtype='object', length=314)

In [36]:
train.columns.to_list()

['applicable_date',
 'elec_prices',
 'expanding_window_price',
 'wdsp_sum_sqrt',
 'wdsp_sqr',
 'wdsp_g',
 'wdsp_c',
 'wdsp_sum',
 'ng_log',
 'daily_natural_gas',
 'ng_sqr',
 'wdsp_sum_log',
 'ng_sqrt',
 'wdsp_d',
 't+1',
 't+2',
 't+3',
 't+4',
 't+5',
 't+6',
 't+7',
 't+8',
 't+9',
 't+10',
 't+11',
 't+12',
 't+13',
 't+14',
 't+15',
 't+16',
 't+17',
 't+18',
 't+19',
 't+20',
 't+21',
 't+22',
 't+23',
 'lag_1_elec_prices',
 'lag_2_elec_prices',
 'lag_3_elec_prices',
 'lag_4_elec_prices',
 'lag_5_elec_prices',
 'lag_6_elec_prices',
 'lag_7_elec_prices',
 'lag_8_elec_prices',
 'lag_9_elec_prices',
 'lag_10_elec_prices',
 'lag_11_elec_prices',
 'lag_12_elec_prices',
 'lag_13_elec_prices',
 'lag_14_elec_prices',
 'lag_15_elec_prices',
 'lag_16_elec_prices',
 'lag_17_elec_prices',
 'lag_18_elec_prices',
 'lag_19_elec_prices',
 'lag_20_elec_prices',
 'lag_21_elec_prices',
 'lag_22_elec_prices',
 'lag_23_elec_prices',
 'lag_24_elec_prices',
 'lag_1_wdsp_sum_sqrt',
 'lag_2_wdsp_sum_sqrt'

In [37]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:14]
lags = cols[37:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[14:37]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['expanding_window_price', 'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_sum', 'ng_log', 'daily_natural_gas', 'ng_sqr', 'wdsp_sum_log', 'ng_sqrt', 'wdsp_d', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_sum_sqrt', 'lag_2_wdsp_sum_sqrt', 'lag_3_wdsp_sum_sqrt', 'lag_4_wdsp_sum_sqrt', 'lag_5_wdsp_sum_sqrt', 'lag_6_wdsp_sum_sqrt', 'lag_7_wdsp_sum_sqrt', 'lag_8_wdsp_sum_sqrt', 'lag_9_wdsp_sum_sqrt', 'lag_10_wdsp_sum_sqrt', 'lag_11_wdsp_sum_sqrt', 'lag_12_wdsp_sum_sqrt', 'lag_13_wdsp_sum_sqrt', 'lag_14_wdsp

In [38]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(248, 289) (248, 24)
(62, 289) (62, 24)


In [39]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [40]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:17.998510



{'mean_fit_time': array([2.38184433, 1.81545851, 2.28975906, 1.15755992, 0.9380553 ,
       1.30703187, 2.24653246, 0.68652399, 0.64414799, 1.54683161]), 'std_fit_time': array([0.13263464, 0.13807191, 0.1784402 , 0.07182063, 0.04911011,
       0.05465292, 0.11984698, 0.07719134, 0.05431286, 0.09875758]), 'mean_score_time': array([0.01981726, 0.02285872, 0.02569065, 0.02109797, 0.02209752,
       0.0194803 , 0.02874601, 0.02358284, 0.02000954, 0.01008604]), 'std_score_time': array([0.00561858, 0.00821287, 0.00966624, 0.00691598, 0.00630632,
       0.00648646, 0.0082797 , 0.00640597, 0.00554375, 0.00705868]), 'param_estimator__subsample': masked_array(data=[0.9, 0.6, 0.7, 0.9, 0.6, 0.5, 0.7, 0.6, 0.6, 0.5],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[100, 90, 90, 60, 50, 70, 100, 30, 30, 100],
  

In [41]:
gbm_top12 = GBM_model(x_train_norm,y_train,0.05,90,9,100,5,10,0.7)

Duration: 0:00:01.105459


In [42]:
y_pred= gbm_top12.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.4159
MSE:  215.3498
RMSE:  14.6748
MAPE:  21.6821
Coefficient of determination (R-squared):  0.0984




(10.415886978445464,
 215.3498067031052,
 14.67480176026597,
 21.682128077365785,
 0.09841122164101178)

In [43]:
XGBM_training(x_train_norm,y_train)

Duration: 0:06:42.258385



{'mean_fit_time': array([22.98348207, 53.47977006, 58.3848386 , 65.75966039, 11.6144599 ,
       37.4559185 , 43.5399157 , 45.96651599, 15.35952818, 84.72123039]), 'std_fit_time': array([0.47798375, 3.54088771, 1.04015621, 2.43904461, 0.37486297,
       2.01514578, 0.93606352, 0.63578619, 0.34376177, 2.61434377]), 'mean_score_time': array([0.0874707 , 0.06700492, 0.06233199, 0.059237  , 0.05588431,
       0.06449542, 0.07263758, 0.05774789, 0.05683591, 0.04334815]), 'std_score_time': array([0.03105101, 0.0068911 , 0.00951154, 0.00984594, 0.0123322 ,
       0.00871344, 0.00692816, 0.01236135, 0.01073976, 0.01045885]), 'param_estimator__subsample': masked_array(data=[0.1, 0.2, 0.30000000000000004, 0.9, 0.1, 0.1, 0.4, 0.8,
                   0.30000000000000004, 0.6],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': mask

In [44]:
xgb_top12 = XGBM_model(x_train_norm,y_train,0.0125,350,7,0.6,0.9)

Duration: 0:00:12.527859


In [45]:
y_pred= xgb_top12.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.3133
MSE:  244.6421
RMSE:  15.641
MAPE:  23.5503
Coefficient of determination (R-squared):  -0.0892




(11.313337802841618,
 244.6420594745692,
 15.641037672564094,
 23.550297706890618,
 -0.0892166316487032)

In [46]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Duration: 0:03:54.955191



{'mean_fit_time': array([34.74526045, 24.01456392, 17.0479949 , 49.10049121, 15.67433457,
       10.07381928, 57.18014379,  7.93658895,  9.1952378 , 39.45730641]), 'std_fit_time': array([0.98531541, 0.92251596, 0.78994298, 1.29176978, 1.44184493,
       0.21916791, 3.76356621, 0.1742846 , 0.15829935, 1.02558102]), 'mean_score_time': array([0.07011542, 0.06444561, 0.044065  , 0.08104503, 0.05913389,
       0.05268357, 0.04289625, 0.04166398, 0.0342694 , 0.03631012]), 'std_score_time': array([0.00925122, 0.03357034, 0.00828775, 0.02053614, 0.012857  ,
       0.03540225, 0.02995476, 0.01023351, 0.00996521, 0.01042014]), 'param_estimator__subsample': masked_array(data=[0.2, 0.8, 0.5, 0.4, 0.1, 0.9, 0.30000000000000004, 0.1,
                   1.0, 0.8],
             mask=[False, False, False, False, 

In [47]:
lgbm_top12 = LGBM_model(x_train_norm,y_train,1229,60,0.0125,200,9,0.5,0.7)

Duration: 0:00:02.415117


In [48]:
y_pred= lgbm_top12.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.503
MSE:  215.2708
RMSE:  14.6721
MAPE:  21.8635
Coefficient of determination (R-squared):  0.0879




(10.50302584768828,
 215.27082761667603,
 14.672110537229333,
 21.863519842402155,
 0.08787167351341041)

### Testing on 2018-2019 data

In [50]:
data_test = data_1819.copy()
data_test = feature_pipeline(data_test,2)
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_sum_sqrt')
data_test = calc_24lags(data_test,'wdsp_sqr')
data_test = calc_24lags(data_test,'wdsp_g')
data_test = calc_24lags(data_test,'wdsp_c')
data_test = calc_24lags(data_test,'wdsp_sum')
data_test = calc_24lags(data_test,'ng_log')
data_test = calc_24lags(data_test,'daily_natural_gas')
data_test = calc_24lags(data_test,'ng_sqr')
data_test = calc_24lags(data_test,'wdsp_sum_log')
data_test = calc_24lags(data_test,'ng_sqrt')
data_test = calc_24lags(data_test,'wdsp_d')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(9686, 315)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_sum', 'ng_log',
       'daily_natural_gas',
       ...
       'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d',
       'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d',
       'lag_22_wdsp_d', 'lag_23_wdsp_d'],
      dtype='object', length=315)


,applicable_date,elec_prices,expanding_window_price,wdsp_sum_sqrt,wdsp_sqr,wdsp_g,wdsp_c,wdsp_sum,ng_log,daily_natural_gas,...,lag_14_wdsp_d,lag_15_wdsp_d,lag_16_wdsp_d,lag_17_wdsp_d,lag_18_wdsp_d,lag_19_wdsp_d,lag_20_wdsp_d,lag_21_wdsp_d,lag_22_wdsp_d,lag_23_wdsp_d
0,2018-09-30 23:00:00,71.267,71.267000,3.206244,41.742968,1.542,3.598,10.280,1.026042,2.79,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
1,2018-10-01 00:00:00,67.212,69.239500,3.285422,45.705908,1.542,4.112,10.794,1.029619,2.80,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
2,2018-10-01 01:00:00,60.500,66.326333,3.285422,45.705908,2.056,3.084,10.794,1.029619,2.80,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
3,2018-10-01 02:00:00,63.682,65.665250,2.867752,28.004776,2.056,1.542,8.224,1.029619,2.80,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
4,2018-10-01 03:00:00,71.617,66.855600,2.584956,16.115956,2.056,1.542,6.682,1.029619,2.80,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14


In [51]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(9663, 315)


False

In [52]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(403, 314)


Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_sum_sqrt', 'wdsp_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_sum', 'ng_log',
       'daily_natural_gas',
       ...
       'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d',
       'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d',
       'lag_22_wdsp_d', 'lag_23_wdsp_d'],
      dtype='object', length=314)

### 10 runs

In [53]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm_top12)

Round 1: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.0298
MSE:  386.2387
RMSE:  19.653
MAPE:  20.3868
Coefficient of determination (R-squared):  0.4564


Round 2: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.2558
MSE:  227.1643
RMSE:  15.072
MAPE:  18.5172
Coefficient of determination (R-squared):  0.4635


Round 3: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.1795
MSE:  501.2242
RMSE:  22.388
MAPE:  19.2716
Coefficient of determination (R-squared):  0.4355


Round 4: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  9.1603
MSE:  199.9965
RMSE:  14.142
MAPE:  17.0079
Coefficient of determination (R-squared):  0.488


Round 5: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  9.608
MSE:  220.0673
RMSE:  14.8347
MAPE:  17.6422
Coefficient of determination (R-squared):  0.5056


Round 6: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.0712
MSE:  337.1613
RMSE:  18.

In [54]:
#XGBM
mae1,mse1,rmse1,mape1,r21 = testing_runs(10,data_test,xgb_top12)

Round 1: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.7273
MSE:  410.3623
RMSE:  20.2574
MAPE:  19.8118
Coefficient of determination (R-squared):  0.546


Round 2: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.2343
MSE:  426.1422
RMSE:  20.6432
MAPE:  19.6742
Coefficient of determination (R-squared):  0.4527


Round 3: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  8.7909
MSE:  247.2419
RMSE:  15.7239
MAPE:  15.8733
Coefficient of determination (R-squared):  0.5737


Round 4: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.6361
MSE:  404.4424
RMSE:  20.1108
MAPE:  18.9457
Coefficient of determination (R-squared):  0.4662


Round 5: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  12.0662
MSE:  424.0322
RMSE:  20.592
MAPE:  21.8494
Coefficient of determination (R-squared):  0.3805


Round 6: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  9.3072
MSE:  280.9943
RMSE: 

In [55]:
#light GBM
mae2,mse2,rmse2,mape2,r22 = testing_runs(10,data_test,lgbm_top12)

Round 1: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.3495
MSE:  331.8921
RMSE:  18.2179
MAPE:  20.646
Coefficient of determination (R-squared):  0.4506


Round 2: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  12.761
MSE:  439.2167
RMSE:  20.9575
MAPE:  22.8815
Coefficient of determination (R-squared):  0.3629


Round 3: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.7396
MSE:  406.6164
RMSE:  20.1647
MAPE:  21.4369
Coefficient of determination (R-squared):  0.4572


Round 4: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.8336
MSE:  351.7951
RMSE:  18.7562
MAPE:  20.8995
Coefficient of determination (R-squared):  0.2704


Round 5: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.1121
MSE:  350.9479
RMSE:  18.7336
MAPE:  20.4644
Coefficient of determination (R-squared):  0.4872


Round 6: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  13.9647
MSE:  644.7413
RMSE

### 30 runs

In [56]:
#GBM
mae3,mse3,rmse3,mape3,r23 = testing_runs(30,data_test,gbm_top12)

Round 1: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  9.1126
MSE:  196.9138
RMSE:  14.0326
MAPE:  16.8628
Coefficient of determination (R-squared):  0.4859


Round 2: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.0464
MSE:  345.2026
RMSE:  18.5796
MAPE:  16.9717
Coefficient of determination (R-squared):  0.4929


Round 3: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.3536
MSE:  282.811
RMSE:  16.817
MAPE:  18.5985
Coefficient of determination (R-squared):  0.5267


Round 4: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.1362
MSE:  285.9428
RMSE:  16.9098
MAPE:  18.2598
Coefficient of determination (R-squared):  0.4886


Round 5: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.8986
MSE:  413.9688
RMSE:  20.3462
MAPE:  18.4648
Coefficient of determination (R-squared):  0.4702


Round 6: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.4464
MSE:  321.9868
RMSE:

In [57]:
#XGBM
mae4,mse4,rmse4,mape4,r24 = testing_runs(30,data_test,xgb_top12)

Round 1: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.5121
MSE:  330.7464
RMSE:  18.1864
MAPE:  18.3626
Coefficient of determination (R-squared):  0.4845


Round 2: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.4119
MSE:  356.7324
RMSE:  18.8874
MAPE:  18.509
Coefficient of determination (R-squared):  0.5218


Round 3: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  9.6989
MSE:  293.7624
RMSE:  17.1395
MAPE:  18.1225
Coefficient of determination (R-squared):  0.457


Round 4: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.7726
MSE:  497.2771
RMSE:  22.2997
MAPE:  19.4124
Coefficient of determination (R-squared):  0.5045


Round 5: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  11.6565
MSE:  366.3575
RMSE:  19.1405
MAPE:  20.1833
Coefficient of determination (R-squared):  0.4375


Round 6: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.3218
MSE:  298.2242
RMSE:

In [58]:
#light GBM
mae5,mse5,rmse5,mape5,r25 = testing_runs(30,data_test,lgbm_top12)

Round 1: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  9.9012
MSE:  248.4666
RMSE:  15.7628
MAPE:  18.8395
Coefficient of determination (R-squared):  0.4746


Round 2: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.574
MSE:  311.266
RMSE:  17.6427
MAPE:  19.6326
Coefficient of determination (R-squared):  0.3976


Round 3: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.7758
MSE:  333.0173
RMSE:  18.2488
MAPE:  19.0422
Coefficient of determination (R-squared):  0.4


Round 4: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.5061
MSE:  269.3987
RMSE:  16.4134
MAPE:  19.2541
Coefficient of determination (R-squared):  0.4012


Round 5: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.1663
MSE:  227.4063
RMSE:  15.08
MAPE:  18.4053
Coefficient of determination (R-squared):  0.432


Round 6: 
(322, 289) (40, 289) (41, 289)
(322, 24) (40, 24) (41, 24)
MAE:  10.9688
MSE:  348.3743
RMSE:  18.6

In [65]:
result_items = pd.DataFrame(columns=["mae", "mse", "rmse", "mape", "r2"])
result_items['mae'] = mae5
result_items['mse'] = mse5
result_items['rmse'] = rmse5
result_items['mape'] = mape5
result_items['r2'] = r25
writer = pd.ExcelWriter("E:/CIT/MSc_AI/Research project_NIMBUS/Data/output.xlsx")
result_items.to_excel(writer, index=False)
writer.close()

In [66]:
import pickle
# save the model to disk
pickle.dump(gbm_top12, open('GBM_top12_model.sav', 'wb'))
pickle.dump(xgb_top12, open('XGBM_top12_model.sav', 'wb'))
pickle.dump(lgbm_top12, open('LightGBM_top12_model.sav', 'wb'))